In [1]:
import sys
import os
from IPython.core.debugger import set_trace
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import numpy as np
import scipy as sci
import csv
import pickle as pk
import h5py
import math as mt
import matplotlib.pyplot as plt
import matplotlib.axis as axis
import matplotlib
import matplotlib.legend as lgd
import sklearn.cluster as cluster
import glob
import six
import time as time
import json
from sklearn import multioutput
import pandas as pd
from pandas.tools.plotting import table
from itertools import groupby
from sklearn.preprocessing import Imputer, normalize, StandardScaler, scale
import sklearn.decomposition as decomp
from scipy.stats import boxcox, mstats, normaltest, pearsonr, skew, iqr
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier
from sklearn.model_selection import cross_val_predict,\
                                    GridSearchCV, KFold,\
                                    permutation_test_score, \
                                    StratifiedKFold, cross_val_score, RandomizedSearchCV
from sklearn.dummy import DummyClassifier, DummyRegressor
import sklearn.feature_selection as fs
from sklearn.linear_model import RandomizedLogisticRegression,\
                                 LogisticRegression,SGDClassifier, Ridge, Lars, ElasticNet
import sklearn.linear_model as lm
from sklearn.neural_network import MLPRegressor
from sets import Set
from sklearn import utils
from collections import OrderedDict
import sklearn.metrics as metrics
from joblib import Parallel, delayed
#import plotly.offline as off
#import plotly.graph_objs as go

from scipy.ndimage.filters import laplace
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache 
from allensdk.api.queries.grid_data_api import GridDataApi
from scipy.ndimage.filters import laplace
import zipfile
import cortical_map_10 as cm
import nrrd
from allensdk.api.queries.ontologies_api import OntologiesApi
import skimage as ski
import rpy2.robjects as ro
import rpy2
import nibabel as nib
import base64
from IPython.core.display import display, HTML
import nrrd

/opt/conda/envs/python2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/opt/conda/envs/python2/lib/python2.7/site-packages/ipykernel_launcher.py:34: DeprecationWarning: the sets module is deprecated


#!pip install allensdk
#!pip install joblib
#! pip install --upgrade "hbp-service-client==1.0.0"
#!conda install -y -c r rpy2 
#!pip install --upgrade sklearn
!pip install --upgrade pip

#clients = get_hbp_service_client()
client      = get_bbp_client().document
collab_path = get_collab_storage_path()
ccp_path    = 'CCP v2'
itemlist = client.listdir(os.path.join(collab_path,ccp_path))
for element in itemlist:
    if os.path.isfile('./' + str(element)) == True: continue
    print element
    local_path = './' + element
    client.download_file(os.path.join(collab_path,ccp_path, element),local_path)
#client.download_file(os.path.join(collab_path, 'cortical_map_10.py'))
#local_path = './cortical_map_10.py'
#client.download_file(os.path.join(collab_path, 'cortical_map_10.py'), local_path)
#local_path = './acr2parent.json'
#client.download_file(os.path.join(collab_path,ccp_path, 'acr2parent.json'),local_path)

## make_data_url function 
### Author: Rembrandt Bakker    
### Description:  
    This function takes as input the name of a nifti volume and converts it to a url   
### Input:  
      filename = the filename of the nifti volume

In [2]:
def make_data_url(filename):
    prefix = 'data:image/gznii;base64,'
    fin = open(filename, 'rb')
    contents = fin.read()
    fin.close()
    #data_url = 'https://scalablebrainatlas.incf.org/composer-dev/?scene={"imports":{"'+ filename +'":{url:"'+ prefix + base64.b64encode(contents)+'",orientation:"PIR"}}}' 
    data_url = '{"imports":{"'+ filename +'":{"url":"'+ prefix + base64.b64encode(contents)+'","orientation":"PIR"}}}' 
    return data_url

## readtable function
### Author: Nestor Timonidis   
### Description:   
Given the file pointer of a .csv file, this function stores the csv elements into a dictionary
*        #### input:    
fp = the file pointer corresponding to the .csv file
*        #### output:   
T = the output dictionary

In [3]:
def readtable(fp):
  r = csv.reader(fp)
  keys = r.next()
  T = [ [] for k in keys ]
  for row in r:
    for i,v in enumerate(row):T[i].append(v)
  T = {keys[i]:col for i,col in enumerate(T)}
  return T

## RemoveOutliers class
### Author: Nestor Timonidis   
### Description:   
Removes outliers based on the input array's per column interquartile range. Elements exceeding the range of $(1.5*iqr - Q1, 1.5*iqr + Q3)$,  
where the interquartile range (iqr) and the two quartiles (1st and 3rd) are defined per column of the array, are detected as outliers, and their corresponding columns are being removed from the array.
*   ### functions:   
       *        #### init:    
initalizes the quartiles used for estimating outliers. By default are Q1 (1st) and Q3 (3rd)
       *        #### fit:   
scans the input array per column and checks for outliers based on the method provided in the description.
For each detected outlier, the column corresponding to it is removed from the array.
The function returns the updated array, as well as a list that stores the outlier indices.


In [4]:
class RemoveOutliers():
     
    def __init__(self, q1 = 25, q2 = 75):
        self.q1 = q1
        self.q2 = q2
    def fit(self, A):
        ranges = iqr(A, axis = 0)
        Q1 = np.percentile(A, self.q1, axis = 0)
        Q3 = np.percentile(A, self.q2, axis = 0)

        genes4deletion = Set()
        for row,cell in enumerate(A):
            for col,gene in enumerate(cell):
               if gene >= ranges[col]*3/2 + Q3[col] or gene <= ranges[col]*3/2 - Q1[col]:
                   genes4deletion.add(col)
                   break

        outliers = list(genes4deletion)
        A = np.delete(A,outliers,1)
        return A,outliers

## filter_cortex function
### Author: Nestor Timonidis   
### Description:   
Given a mask corresponding to specific stuctures of the mouse cortex and a tract tracing experiment defined in a volumetric space, this function filters out voxels of the experiment not corresponding to the mask.
*        #### input:    
         * struct_idx_dict = dictionary that maps identities of anatomically distinct structures, as provided by the Allen Institute, to their position in a unionized space, meaning a 2D space of anatomical structures and brain related experiments.
         * l_vals = the anatomical structures that will be used to filter the data, defined by their position in the unionized space.
         * Annotation = the Annotation identities of the structures in the volumetric space, as provided by the Allen Institute.
         * pd: the tract tracing experiment defined in the volumetric space, measured by projection density.
*        #### output:   
         * l_voxel_mask = the refined volumetric data based on the applied mask.

In [5]:
def filter_cortex(struct_idx_dict,l_vals, Annotation, pd):
        scis = [sci for sci,x in struct_idx_dict.items() if x in l_vals]
        l_mask = []
        for sci in scis:
            tmp = Annotation == int(sci) 
            if l_mask == []:
                l_mask = tmp
            else:
                l_mask = np.ma.mask_or(l_mask,tmp)   

        l_voxel_mask = np.zeros(np.shape(pd))
        l_nzero      = np.nonzero(l_mask)
        l_voxel_mask[l_nzero] = pd[l_nzero]
        return l_voxel_mask

## MakeArrayBorders function
### Author: Nestor Timonidis   
### Description:   
This function creates a structural border mask given the annotation volume of the mouse brain, based on an anatomical parcelation made by the Allen Institute. The voxels are masked based on the ones that differ in value with their neighboring voxels and the rest.
*        #### input:    
         * Annotation_copy = annotation volume of the mouse brain (defined in micrometers).
*        #### output:   
         * Annotation_copy2 = the structural border mask

In [6]:
def MakeArrayBorders(Annotation_copy):
    #description: given a matrix, comparison with neighbors so that different neighboring pixels get the value 1 
    #             and create borders, else 0
    
    Annotation_copy2 = np.zeros(np.shape(Annotation_copy))
    
    for x,row in enumerate(Annotation_copy):
       
        for y,col in enumerate(row):
            flag = 0
            if x > 0:
                if (Annotation_copy[x,y] - Annotation_copy[x-1,y]) != 0:
                    flag = 1
            if x < np.shape(Annotation_copy)[0] - 1:
                if (Annotation_copy[x,y] - Annotation_copy[x+1,y]) != 0:
                    flag = 1
            if y > 0:
                if (Annotation_copy[x,y] - Annotation_copy[x,y-1]) != 0:
                    flag = 1
            if y < np.shape(Annotation_copy)[1] - 1:
                if (Annotation_copy[x,y] - Annotation_copy[x,y+1]) != 0:
                    flag = 1    
                    
            if flag == 1:
                Annotation_copy2[x,y] = 1
            else:
                Annotation_copy2[x,y] = 0
                
    return Annotation_copy2          

## BrainPlotter class
### Author: Nestor Timonidis   
### Description:   
This class contains functions that provide visualizations of the mouse brain, either in the form of brain slices or cortical flatmaps. The inputs of interest for visualization are identities of tract tracing or gene expression experiments, as defined by the Allen Institute.
*   ### functions:   
       *        #### init:  
                *  #### description:  
                creates a brain plotter object by defining a number of parameters for the visualization process.
                *  #### input:  
                      *  mcc = an instance of MouseConnectivityCache class
                      *  experiment_id = the identity of the experiment  
                      *  driver  =  the type of the experiment. It could be wild_type (tract tracing), or gene_expression for the equivalent experiment categories, or the name of a cre category (tract tracing).
                      * res = resolution of the experiment in micrometers. Acceptable options are: 100, 25 and 10 for tract tracing experiments, and 200 for gene expression.
                     
       *        ####   plot_slice:   
                * #### description:   
                provides a visualization in the form of a brain slice, given the parameters defined by the init function and a set of other parameters. This is done by merging the experimental data together with a registration nissl template in an rgb form.
                 *  #### input:  
                       * matrix = a 3-dimensional array corresponding to an experiment as a manual option. Default = 'None'.
                       * struct_idx_dict = dictionary that maps identities of anatomically distinct structures, as provided by the Allen Institute, to their position in a unionized space, meaning a 2D space of anatomical structures and brain related experiments.
                       * l_vals = the anatomical structures that will be used to filter the data, defined by their position in the unionized space, in case they are being provided.  
                       * Annotation = the Annotation identities of the structures in the volumetric space, as provided by the Allen Institute.
                       * slice_idx = the point in the first dimension of the data (anterior-posterior range), which will be stable for the brain slice to be visualized. Default = 264.
                       * savefile = the save file name. Default = 'slice.png'.
         *        #### plot_flatmap:  
                  * #### description:   
                 provides a visualization in the form of a cortical flatmap, given the parameters defined by the init function and a set of other parameters. This is done by converting the experimental data in a flatmap form (calling the cortical_map_10 function), and then merging them together with a registration nissl template and an annotation template in an rgb form.
                 *  #### input:  
                           * matrix = a 3-dimensional array corresponding to an experiment as a manual option. Default = 'None'
                           * l_vals = the anatomical structures that will be used to filter the data, defined by their position in the unionized space, in case they are being provided.  
                           * Annotation = the Annotation identities of the structures in the volumetric space, as provided by the Allen Institute.
                           * savefile = the save file name. Default = 'slice.png'.
                 *  ### output:  
                           * trs_pd = the cortical flatmap in a rgb form.
                           * mask_and_pd = the cortical flatmap merged with with a registration nissl template and an annotation template in an rgb form.
                 

In [7]:
class BrainPlotter():
    
    def __init__(self, mcc, experiment_id = 181599674, driver = 'wild_type', res = 25):
        
        self.id     = experiment_id
        self.driver = driver  
       
        if driver != 'gene_expression':
            flag = 0 
            # projection density: number of projecting pixels / voxel volume
            all_files = os.listdir('./')
            experiment_list = [val for val in all_files if 'experiment_' in val]
            file_to_look  = 'experiment_' + str(experiment_id)
            if file_to_look in experiment_list:
                pathway  = os.path.join('./',file_to_look)
                pd_files = os.listdir(pathway)
                if 'projection_density_' + str(res) + '.nrrd' in pd_files:
                    self.pd, pd_info = nrrd.read(pathway +  '/projection_density_' + str(res) + '.nrrd') 
                else:
                    self.pd, pd_info = mcc.get_projection_density(experiment_id)

            else:
               self.pd, pd_info = mcc.get_projection_density(experiment_id)
            
        else:
    
            GDA = GridDataApi(resolution = res)
            #GDA.download_expression_grid_data(experiment_id, include = ['energy','density','intensity'], path = 'local.zip')
            GDA.download_gene_expression_grid_data(experiment_id, GDA.INTENSITY, path = 'local.zip')
            zip_ref = zipfile.ZipFile('local.zip', 'r')
            zip_ref.extractall('expression_files')
            zip_ref.close()
            with open('expression_files/density.raw','rb') as fp:
                buffer = fp.read()
            
            dims = [67,41,58]
            A  = np.ndarray(shape = dims, dtype = np.float32, buffer = buffer, order = 'F')
            fp.close()
            self.xd = A
        
        # For reducing time complexity, the brain slice templates provided by the Allen Institute's 
        # MouseConnectivityCache tool have already been uploaded and stored as files:
        # average_template_10.nrrd and average_template_25.nrrd. The code commented bellow
        # displays how the templates can be downloaded in a custom fashion.
        #self.template, template_info = mcc.get_template_volume()
        self.template, template_info = nrrd.read('average_template_'+ str(res) + '.nrrd')
  

    def plot_slice(self,struct_idx_dict,l_vals, Annotation, matrix = 'None', slice_idx = 264, savefile = 'slice.png'):    
        
        #*** Laplacian intervention ****#
        if l_vals == 'None':
            l_voxel_mask = self.pd
        else:            
            l_voxel_mask = filter_cortex(struct_idx_dict,l_vals, Annotation, self.pd)
            
        #1:
        mask_lpc = np.zeros((np.shape(self.template)[1],np.shape(self.template)[2]))
        laplace(self.template[slice_idx,:,:], mask_lpc)
        
        norm_mask_lpc = mask_lpc/100
        norm_mask_lpc[norm_mask_lpc > 1] = 1
        
        cmap_mask = plt.get_cmap('gray') 
        rgb_mask = cmap_mask(norm_mask_lpc, bytes = True)
      
        
        if self.driver != 'gene_expression':
            # compute the maximum intensity projection (along the anterior-posterior axis) of the projection data
            if matrix != 'None':
                pd_mip = matrix[slice_idx,:,:]
            else:                
                pd_mip  = l_voxel_mask[slice_idx,:,:]
            
            pd_mip  = pd_mip/(1.0*np.max(pd_mip))
            #3:
            cmap_pd  = plt.get_cmap('hot') 
            rgb_pd  = cmap_pd(pd_mip, bytes = True)
            #4:
            new_mat  = np.maximum(rgb_pd, rgb_mask)
            
        else:
            xd_mip  = self.xd[slice_idx,:,:]
            xd_mip  = ski.transform.rescale(xd_mip, (320/41.0,456/58.0))     #(528,320,456)
            cmap_xd = plt.get_cmap('hot') 
            rgb_xd  = cmap_xd(xd_mip,bytes = True)
            new_mat = np.maximum(rgb_xd, rgb_mask)
        
        # Look at a slice from the average template and annotation volume
        fig  = plt.figure(figsize=(15, 6))
        plt.imshow(new_mat, cmap = 'hot', aspect='equal', vmin = 0, vmax = 1)
        plt.title("registration template for: " + self.driver, fontsize = 20)
        plt.savefig(savefile)
        plt.show()
    
    def plot_flatmap(self,l_vals, Annotation, matrix = 'None', savefile = 'flatmap.png'):  
        
        cm1=cm.CorticalMap(projection='top_view')
        cm2=cm.CorticalMap(projection='dorsal_flatmap')
        
        if l_vals == 'None': 
            l_voxel_mask = self.pd
        else:       
            l_voxel_mask = filter_cortex(struct_idx_dict,l_vals, Annotation, self.pd)

        
        #%% scale the nissl template
        #%% initialize colormaps for rgb conversion
        self.template = self.template/(1.0*np.max(self.template))
        cmap_mask     = plt.get_cmap('gray') 
        cmap_pd       = plt.get_cmap('hot') 
        #%% Process the annotation template
        #Annot_lpc     = np.zeros(np.shape(Annotation))
        #laplace(Annotation,Annot_lpc)
        #Annot_lpc = Annot_lpc/100
        #Annot_lpc[Annot_lpc > 1] = 1
        trs_annot_lpc = cm1.transform(Annotation, agg_func = np.mean)
        trs_annot_lpc_bordered = MakeArrayBorders(trs_annot_lpc)
        pk.dump(trs_annot_lpc_bordered,open('Annotation_copy2.pkl','wb'))
        #trs_annot_lpc_bordered = pk.load(open('Annotation_copy2.pkl','rb'))
        rgb_annot     = cmap_mask(trs_annot_lpc_bordered, bytes = True)
        #%% transform projection density and nissl template
                
        if matrix != 'None':
            trs_pd = cm1.transform(matrix, agg_func = np.mean)
        else:    
            trs_pd = cm1.transform(l_voxel_mask, agg_func = np.mean)
            
        trs_mask      = cm1.transform(self.template, agg_func = np.mean)
        #%% convert projection density and nissl template to rbg images
        rgb_mask      = cmap_mask(trs_mask, bytes = True)
        rgb_pd        = cmap_pd(trs_pd, bytes = True)
        #%% mix rgb images - projection density with nissl template and annotation
        mask_and_pd   = np.maximum(rgb_pd,rgb_mask)
        annot_and_pd  = np.maximum(rgb_annot,rgb_pd)
        annot_mask_pd = np.maximum(mask_and_pd,rgb_annot)
        pk.dump(annot_mask_pd, open(str(savefile) + '_annot_mask.pkl','wb'))
        pk.dump(annot_and_pd, open(str(savefile) + '_annot_and_pd.pkl','wb'))
        #%%
        fig = plt.figure
        plt.imshow(annot_and_pd)
        savepoint1 = savefile + '_nonnissl.jpg' 
        plt.savefig(savepoint1, dpi = 2000, format = 'jpg')
        plt.show()
        
        savepoint2 = savefile + '_nissl.jpg'
        plt.figure
        plt.imshow(annot_mask_pd)
        plt.savefig(savepoint2, dpi = 2000, format = 'jpg')
        plt.show()
       
        return trs_pd, mask_and_pd
        

## FromVoxel2Union function
### Author: Nestor Timonidis   
### Description:   
This function maps the projection patterns that are defined by a given tract tracing experiment across anatomically distinct brain areas (unionized space), to a volumetric 3D space as specified by an annotation volume provided by the Allen Institute.
*        #### Input:  
         * Annotation = the Annotation identities of the structures in the volumetric space, as provided by the Allen Institute.
         * struct_idx_dict = dictionary that maps identities of anatomically distinct structures, as provided by the Allen Institute, to their position in a unionized space, meaning a 2D space of anatomical structures and brain related experiments.
         * ConStr: the tract tracing experiment defined in the unionized space, measured by normalized projection density
         * Pred_ConStr: the equivalent tract tracing experiment as ConStr, but with values being predicted by the the gene expression based predictive model (MesoPred).
*        #### Output:   
         * UnionToVoxel      =  a 3D array containing the tracing data of the original experiment mapped to the volumetric space.
         * UnionToVoxel_pred =  a 3D array containing the tracing data of the predicted experiment mapped to the volumetric space.

In [8]:
def FromUnion2Voxel(Annotation, struct_idx_dict, ConStr, Pred_ConStr = 'None'):
    
    original_shape       = Annotation.shape
    Annotation_flat      = np.ndarray.flatten(Annotation,order = 'C')
    UnionToVoxel         = np.zeros(np.shape(Annotation_flat))
    UnionToVoxel_pred    = np.zeros(np.shape(Annotation_flat))
    rem_ind              = np.asarray(params['remaining_indices'])
    i = 0
    for area_id,area_num in struct_idx_dict.items():
        i += 1
        if i%50 == 1:
            print i
        if int(area_id) in Annotation_flat:
            area_mask = np.nonzero(Annotation_flat == int(area_id))
            #area_num_reind = [idx for idx,val in enumerate(params['remaining_indices']) if val == area_num]
            if area_num in rem_ind:
                area_num_reind = rem_ind == area_num
                union_values                 = ConStr[area_num_reind]
                union_values2                = Pred_ConStr[area_num_reind]
                UnionToVoxel[area_mask]      = union_values
                UnionToVoxel_pred[area_mask] = union_values2
  
    UnionToVoxel      = UnionToVoxel.reshape(original_shape, order = 'C')
    UnionToVoxel_pred = UnionToVoxel_pred.reshape(original_shape, order = 'C')
    
    return UnionToVoxel, UnionToVoxel_pred

## BinarizeTheVector function
### Author:   Nestor Timonidis    
### Description:   
Given an input vector with
continuous values, the function binarizes
its values based on a threshold value.     
* ### Input:     
     * input_vec = the vector to be binarized,  
     * binThreshold = the desired ratio between the positive and negative class    
* ### Output:   
    * y = the binarized vector.

In [9]:
def BinarizeTheVector(input_vec,binThreshold):
    percentile = (1-binThreshold)*100
    Q = np.percentile(input_vec, percentile, axis = 0)
    y = np.zeros(np.shape(input_vec))
    for idx,val in enumerate(input_vec):
        if input_vec[idx] > Q:
            y[idx] = 1   
    return y 

## CustomCrossval function
### Author:   Nestor Timonidis    
### Description:   
A custom implementation of the cross-validation method, that validates the performance of a supervised learning model by partitioning into disjoint training and testing sets and testing each test set by the model trained by the train set. The difference of this implementation with the custom ones provided by sklearn is that this implementation stores a set of additional parameters at once, as described by the output section. the actual and the predicted values in corresponding lists, stores class , the model parameters and the model coefficients assigned to the trained features based on averaging over the training sets.    
* ### Input:    
     * mdl = the supervised learning model for utility.
     * X = the input dataset or set of independent variables.   
     * y = the dataset labels or dependent variables.
     * cv = the cross validation method as defined by sklearn. 
* ### Output:   
    * probs = assignment probabilities in case of support by the model.
    * preds = list with all predicted data points, assigned to the list based on their corresponding index in the actual data.
    * trues = the list of the actual samples, on one to one correspondance with the preds list.
    * mdl_list = storage of all model parameters.
    * mean_coeffs = the coefficients attributed to the data features, averaged across all training sets.

In [10]:
def CustomCrossval(mdl,X,y,cv):
    
    probs = np.zeros((np.shape(y)[0],2)); preds = np.zeros((np.shape(y)));
    trues = np.zeros((np.shape(y)))
    coeffs = []; mdl_list = []
    for train, test in cv.split(X,y):
        mdl.fit(X[train],y[train])
        mdl_list.append(mdl)
        if hasattr(mdl, 'best_estimator_'):
            if hasattr(mdl.best_estimator_, 'coef_'):
                coeffs.append(mdl.best_estimator_.coef_)
            elif hasattr(mdl.best_estimator_, 'feature_importances_'):
                coeffs.append(mdl.best_estimator_.feature_importances_)
        if hasattr(mdl, 'predict_proba'):        
            probs[test,:] = mdl.predict_proba(X[test])
        preds[test] = mdl.predict(X[test])
        trues[test] = y[test]
    mean_coeffs = np.mean(coeffs,0)        
    return probs, preds, trues, mdl_list, mean_coeffs

## RemoveNanStructs class
### Author:   Nestor Timonidis    
### Description:   
Given a dataset as input, this function removes columns or rows containing NaN (not-a-number), based on whether the percentage of NaN exceeds a given threshold.
*   ### functions:   
       *        #### init:    
                *  #### description:  
                initializes a RemoveNanStructs class object by selecting the default dimension of the dataset for NaN scanning, and the threshold for exclusion.
                *  #### input:  
                      *  dim = dimension for selection.
                      *  nan_thr = the exclusion threshold.
       *        ####   fit:   
                * #### description:   
                Performs the exclusion of rows or columns as described above.
                 *  #### input:  
                       * X = the dataset to be processed.

                 *  #### output:  
                       * X = the dataset after being processed.
                       * nan_list = list containing the rows or columns that were excluded.

In [11]:
class RemoveNanStructs():
    def __init__(self, dim = 0, nan_thr = 0.1):
        self.nan_thr = nan_thr
        self.dim = dim
    def fit(self,X):
        nan_list = []
        if self.dim == 0:    #row
            for idx,row in enumerate(X):
                nancols = [col for col in row if mt.isnan(col) == True]
                if len(nancols) >= len(row)*self.nan_thr: # row for deletion spotted
                    nan_list.append(idx)
        elif self.dim == 1:   #column
            for idy in range(len(X[0])):
                nanrows = [X[idx,idy] for idx in range(len(X)) if mt.isnan(X[idx,idy]) == True]
                if len(nanrows) >= len(X[0])*self.nan_thr: # column for deletion spotted
                    nan_list.append(idy)
        X = np.delete(X,nan_list,self.dim)
        return X,nan_list

## BimodalImputation class
### Author:   Nestor Timonidis    
### Description:   
Given a vector that is partitioned into two classes, imputes its missing values by sampling from the two classes according to their distribution.   
*   ### functions:   
       *        #### init:    
                *  #### description:  
                initializes a BimodalImputation class object by selecting the default class for selection as defined by mode.
                *  #### input:  
                      *  mode =  the default class for selection. Default = 0.
                     
       *        ####   fit:   
                * #### description:   
                Performs the bimodal imputation as described above.
                 *  #### input:  
                       * y = the vector for imputation.

                 *  #### output:  
                       * y = the imputed vector

In [12]:
class BimodalImputation():
    def __init__(self, mode = 0):
         self.mode = mode
    def fit(self, y):
        modality = {}
        nan_poses = [idx for idx,val in enumerate(y) if mt.isnan(val) == True]
        modality[0] = [idx for idx,val in enumerate(y) if val == self.mode]
        modality[1] = [idx for idx,val in enumerate(y) if val != self.mode and mt.isnan(val) == False]
        freq    = np.zeros((2,1))
        freq[0] = len(modality[0])/float(len(y))
        freq[1] = len(modality[1])/float(len(y))
        max_pos = [idx for idx in range(2) if freq[idx] == max(freq)][0]
        min_pos = [idx for idx in range(2) if freq[idx] == min(freq)][0]

        for nan in nan_poses:   # rinse and repeat for all nans
            # Draw a dice
            dice = np.random.random_sample()
            if dice <= min(freq):
               rnd_sl = np.random.randint(low = 0, high = len(modality[min_pos]))
               sample = modality[min_pos][rnd_sl]
            else:
               rnd_sl = np.random.randint(low = 0, high = len(modality[max_pos]))
               sample = y[modality[max_pos][rnd_sl]]

            y[nan] = sample

        return y

## Save2Nifti function 
### Author: Nestor Timonidis    
### Description:
This function takes a voxelized gridded matrix and creates nifti volumes out of each columns, as well as converts 
    the volumes to urls in order to be visualized in the Scalable Brain Composer 3D brain visualization tool.   
* ### Input:  
    * brainData = the voxelized gridded matrix - contains predictions of the connection between each grid and a number of target brain areas. 
* ### Output:    
    * nifti_urls = the urls of the produced nifti volumes.

In [13]:
def Save2Nifti(brainData):
    
    nifti_urls = {}
    print 'Saving results to nifti form ............... \n'
    Affine = np.eye(4)
    outfile = 'volume_density'
    for row in range(3):
        Affine[row][row] = 0.2
    for experiment,p_density in brainData.items():
        density_img = nib.Nifti1Image(p_density, Affine)
        outfile_revisited = outfile + '_'+ str(experiment) + '.nii.gz'
        nib.save(density_img, outfile_revisited)
        nifti_urls[experiment] = make_data_url(outfile_revisited)
       
    print 'Finished converting nifti files to url form ............... \n'    
    return nifti_urls

## render_mpl_table function
### author: Nestor Timonidis
### description:   
This function renders for visualizing data in a table form 

In [14]:
def render_mpl_table(data, col_width = 13.0, row_height = 0.625, font_size=14,
                     header_color='#40466e', row_colors=['#f1f1f2', 'w'], edge_color='w',
                     bbox=[0, 0, 1, 1], header_columns=0,
                     ax=None, **kwargs):
    if ax is None: 
        size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
        fig, ax = plt.subplots(figsize=size)
        ax.axis('off')

    mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns, **kwargs)

    mpl_table.auto_set_font_size(False)
    mpl_table.set_fontsize(font_size)

    for k, cell in  six.iteritems(mpl_table._cells):
        cell.set_edgecolor(edge_color)
        if k[0] == 0 or k[1] < header_columns:
            cell.set_text_props(weight='bold', color='w')
            cell.set_facecolor(header_color)
        else:
            cell.set_facecolor(row_colors[k[0]%len(row_colors) ])
    return ax

## MesoconnectomePredictor class
### Author: Nestor Timonidis
### Description:   
This class nests a set of function that are involved in the mesoconnectome prediction pipeline.
*   ### init function
    *   ### description:   
    initalizes a MesoconnectomePredictor object by specifying the gene expression (GeneExp) and connectivity strength (ConStr) matrices for analysis.



In [15]:
class MesoconnectomePredictor:

    def __init__(self, GeneExp = [], ConStr = []):
        self.GeneExp = GeneExp
        self.ConStr = ConStr

class MesoconnectomePredictor(MesoconnectomePredictor):    
    
    def GO2Csv(self, BP, MF, CC, finalhits, ont_hits, gene_list, gene_stars):
        # Desciption: Given GOenrichment analysis results, appends them to a .csv file
        BP = list(np.asarray(BP))[0]; MF = list(np.asarray(MF))[0]; CC = list(np.asarray(CC))[0]
        write_to = os.path.join(params['prefix'],'hide_the_csvs', str(params['cre-line']) + '_GOenrichment_results')
         
        GODict = OrderedDict()
        GODict['BIOLOGICAL PROCESS'] = [element for element in BP]
        GODict['MOLECULAR FUNCTION'] = [element for element in MF]
        GODict['CELLULAR COMPONENT'] = [element for element in CC]
        GODict['CELL TYPE MARKERS']  = [key + ' : ' + str(val) for key,val in finalhits.items()]
        GODict['CELL TYPE ONTOLOGIES']       = [key + ' : ' + str(val) for key,val in ont_hits.items()]
        GODict['TOP ' + str(gene_stars) + ' GENES']  = [gene for gene in gene_list]
        
        GO_DF = pd.DataFrame(dict([ (k, pd.Series(v)) for k,v in GODict.items() ])) 
        tbl = render_mpl_table(GO_DF)
         
        plt.plot(frame_on = False) # no visible frame
        ax = plt.gca()
        ax.xaxis.set_visible(False)  # hide the x axis
        ax.yaxis.set_visible(False)  # hide the y axis
        table(ax, GO_DF)
        plt.savefig(write_to + '.png')
        plt.show()
        write_to = write_to + '.csv'    
        '''PM.writetable(['BIOLOGICAL PROCESS'], open(write_to, 'wb'))
        PM.writetable(BP, open(write_to, 'ab'))
        PM.writetable([''], open(write_to, 'ab'))
        PM.writetable(['MOLECULAR FUNCTION'], open(write_to, 'ab'))
        PM.writetable(MF, open(write_to, 'ab'))
        PM.writetable([''], open(write_to, 'ab'))
        PM.writetable(['CELLULAR COMPONENT'], open(write_to,'ab'))
        PM.writetable(CC, open(write_to, 'ab'))
        PM.writetable([''], open(write_to, 'ab'))
        PM.writetable(['CELL-TYPE ONTOLOGIES'], open(write_to,'ab'))
        PM.writetable(finalhits, open(write_to, 'ab'))
        PM.writetable([''], open(write_to, 'ab'))
        PM.writetable(['CAHOY ONTOLOGIES HITS'], open(write_to,'ab'))
        PM.writetable(ont_hits, open(write_to, 'ab'))
        PM.writetable([''], open(write_to, 'ab'))
        PM.writetable(['TOP ' + str(gene_stars) + ' GENES'], open(write_to,'ab'))
        PM.writetable(gene_list, open(write_to, 'ab'))'''

## MesoconnectomePredictor class
### GOenrichment function
### author: Paul Tiesinga, Nestor Timonidis
### description:  
Given a set of genes, this function performs gene ontology enrichment analysis by comparing the set with the org.Mm.eg.db mouse gene database provided by https://bioconductor.org/packages/release/data/annotation/html/org.Mm.eg.db.html. The script utilizes a nested script written in the R programming languages, using the rpy2 library
* ### input:
    * GeneList = the list of genes
* ### output:
    * BP = biological processes related to the gene list based on their statistical significance as indicated by the analysis
    * MF = molecular functions related to the gene list based on their statistical significance as indicated by the analysis
    * CC = cellular components related to the gene list based on their statistical significance as indicated by the analysis

In [16]:
class MesoconnectomePredictor(MesoconnectomePredictor):

    def GOenrichment(self, GeneList):
        # Description: given a set of selected genes, performs enrichment
        #              analysis comparing to a bioconductor database
        ro.numpy2ri.activate()
        MGP = ro.r('''
        GOenrichment <-function(GeneList){
            source("http://bioconductor.org/biocLite.R")
            biocLite("ALL")
            biocLite("GOstats")
            biocLite("Category")
            biocLite("genefilter")
            biocLite("org.Mm.eg.db")
            biocLite("gage")
            biocLite(pkgs=c("Biobase", "IRanges", "AnnotationDbi"),
                suppressUpdates=FALSE,
                suppressAutoUpdate=FALSE,
                siteRepos=character(),
                ask=TRUE)
            #install.packages("RSQLite")
            #install.packages("devtools")
            #require(devtools)
            library("RSQLite")
            library("gage")
            library("genefilter")
            library("org.Mm.eg.db")
            library(DBI)
            library("GO.db")
            library("GOstats")
            library("Category")
            library("annotate")
            data(ALL, package="ALL")

            hgCutoff <- 0.001
            params <- new("GOHyperGParams",
                   geneIds = GeneList,
                   universeGeneIds = NULL,
                   annotation = "org.Mm.eg.db",
                   ontology = "BP",
                   pvalueCutoff = hgCutoff,
                   conditional = FALSE,
                   testDirection = "over")
            BP  <- hyperGTest(params)
            sumBP <- data.frame(summary(BP))
            sumBP <- subset(sumBP, select=c("Term"))

            params <- new("GOHyperGParams",
                   geneIds = GeneList,
                   universeGeneIds = NULL,
                   annotation = "org.Mm.eg.db",
                   ontology = "MF",
                   pvalueCutoff = hgCutoff,
                   conditional = FALSE,
                   testDirection = "over")
            MF <- hyperGTest(params)
            sumMF <- data.frame(summary(MF))
            sumMF <- subset(sumMF, select=c("Term"))

            params <- new("GOHyperGParams",
                   geneIds = GeneList,
                   universeGeneIds = NULL,
                   annotation = "org.Mm.eg.db",
                   ontology = "CC",
                   pvalueCutoff = hgCutoff,
                   conditional = FALSE,
                   testDirection = "over")
            CC <- hyperGTest(params)
            sumCC <- data.frame(summary(CC))
            sumCC <- subset(sumCC, select=c("Term"))

            foo <- vector(mode="list", length = 3)
            foo[[1]] <- sumBP
            foo[[2]] <- sumMF
            foo[[3]] <- sumCC

            return(foo)
        }''')
        r_getname = ro.globalenv['GOenrichment']
        verdict = r_getname(GeneList)
        BP = verdict[0]
        MF = verdict[1]
        CC = verdict[2]

        return BP, MF, CC

## MesoconnectomePredictor class
### PlotTheResults function
### author: Nestor Timonidis
### description:  
This function plots and saves the results of the predictive analysis.
* ### input:
    * Results  = the analysis results
    * MetaInfo = information related to the plots. Examples include the size of the plot ticks and the filename for save.


In [17]:
class MesoconnectomePredictor(MesoconnectomePredictor):

    def PlotTheResults(self,Results,MetaInfo):

            k = len(MetaInfo['xtick'])
            fsz = 9
            fig = plt.figure(figsize=(8,6))

            #plt.tight_layout()
            plt.rcParams['figure.figsize']
            plt.boxplot(Results, 0, 'gD', widths = 0.6, whis = [5,95])

            ax = plt.gca()
            ax.xaxis.set_tick_params(labelsize = 14)
            ax.yaxis.set_tick_params(labelsize = 14)
            plt.xticks([i+1 for i in range(k)], MetaInfo['xtick'],rotation = 0)
            plt.text(0.55, MetaInfo['max1'][1], MetaInfo['max1'][0], fontsize = fsz)
            plt.text(0.55, MetaInfo['min1'][1], MetaInfo['min1'][0], fontsize = fsz)
            plt.text(0.55, MetaInfo['med1'][1], MetaInfo['med1'][0], fontsize = fsz)
            plt.text(1.45, MetaInfo['max2'][1], MetaInfo['max2'][0], fontsize = fsz)
            plt.text(1.45, MetaInfo['min2'][1], MetaInfo['min2'][0], fontsize = fsz)
            plt.text(1.45, MetaInfo['med2'][1], MetaInfo['med2'][0], fontsize = fsz)
            plt.text(2.45, MetaInfo['max3'][1], MetaInfo['max3'][0], fontsize = fsz)
            plt.text(2.45, MetaInfo['min3'][1], MetaInfo['min3'][0], fontsize = fsz)
            plt.text(2.45, MetaInfo['med3'][1], MetaInfo['med3'][0], fontsize = fsz)
            plt.title(MetaInfo['title'], fontsize = 14)
            # plt.xlabel(MetaInfo['xlabel'])
            plt.ylabel(MetaInfo['ylabel'], fontsize = 14)
            plt.yticks(np.arange(MetaInfo['lb'] - np.std(Results), MetaInfo['ub'] + np.std(Results), np.std(Results)))
            #plt.yticks(np.arange(0.4,1.0,0.05))

            plt.savefig(MetaInfo['save_file'])
            plt.show(block = False)
            plt.pause(1)
            plt.close()

## MesoconnectomePredictor class
### Evaluation function
### author: Nestor Timonidis
### description:  
This function plots and saves the results of the predictive analysis.
* ### input: 
    * ClfResults_lr = results related to a linear model (classifier or regressor).
    * ClfResults_rf = results related to a random forest model (classifier or regressor).
    * ClfResults_bl = results related to a control (baseline) model (classifier or regressor).
    * params = parameters related to the evaluation of the analysis. Examples include the abbreviations of the brain structures.

In [18]:
 class MesoconnectomePredictor(MesoconnectomePredictor):   
    
    def Evaluation(self,ClfResults_lr,ClfResults_rf, ClfResults_bl, params):
 
          MetaInfo = {}
          templ = 'values for predicted tracer - projection patterns\nof driver line:'
          MetaInfo['xlabel'] = 'Classifier Models'
          exclusions = ['Model Storage', 'Gene Scoring', 'Mean Gene Scoring']
          for measure in ClfResults_lr.keys():
             if measure not in exclusions:
               tmp1 = [val[0] for val in ClfResults_lr[measure]] 
               tmp2 = [val[0] for val in ClfResults_rf[measure]] 
               tmp3 = [val[0] for val in ClfResults_bl[measure]] 
               sort1 = np.argsort(tmp1)[::-1]; sort3 = np.sort(tmp1)[::-1]
               sort2 = np.argsort(tmp2)[::-1]; sort4 = np.sort(tmp2)[::-1]
               sort5 = np.argsort(tmp3)[::-1]; sort6 = np.sort(tmp3)[::-1]
               set_length = len(sort1)   
            
               MetaInfo['min1']  = [params['structure-abbrev'][sort1[set_length-1]], sort3[set_length-1]]
               MetaInfo['max1']  = [params['structure-abbrev'][sort1[0]], sort3[0]]
               MetaInfo['med1']  = [params['structure-abbrev'][sort1[set_length/2]], sort3[set_length/2]]
    
               MetaInfo['min2']  = [params['structure-abbrev'][sort2[set_length-1]], sort4[set_length-1]]
               MetaInfo['max2']  = [params['structure-abbrev'][sort2[0]], sort4[0]]
               MetaInfo['med2']  = [params['structure-abbrev'][sort2[set_length/2]], sort4[set_length/2]]
               MetaInfo['min3']  = [params['structure-abbrev'][sort5[set_length-1]], sort6[set_length-1]]
               MetaInfo['max3']  = [params['structure-abbrev'][sort5[0]], sort6[0]]
               MetaInfo['med3']  = [params['structure-abbrev'][sort5[set_length/2]], sort6[set_length/2]] 
               
               MetaInfo['save_file'] = params['cre-line'] + '_' + measure +'.png'
               MetaInfo['title']     = params['cre-line'] #measure + ' '+ templ + ' ' + params['cre-line']
               MetaInfo['ylabel']    = measure
               MetaInfo['xtick']     = ['Ridge Regression','Random Forest', 'Control']
               if measure == 'AURoc': 
                  MetaInfo['lb'] = 0.4
                  MetaInfo['ub'] = 1.0
               elif measure == 'RMSE' or measure == 'r2': 
                  MetaInfo['lb'] = min([sort3[set_length-1],sort4[set_length-1],sort6[set_length-1]])
                  MetaInfo['ub'] = max([sort3[0],sort4[0],sort6[0]])
               else: 
                  MetaInfo['lb'] = 0.1 
                  MetaInfo['ub'] = 1.0
                
               JoinResults = np.asarray([(a[0],b[0],c[0]) for a,b,c in zip(ClfResults_lr[measure],ClfResults_rf[measure],ClfResults_bl[measure])])
               MesoPred.PlotTheResults(JoinResults,MetaInfo)  
          
          '''gene_stars =  50
          BP, MF, CC = MesoPred.GOenrichment(ClfResults_lr['Mean Gene Scoring'][0][0:gene_stars])
          GODict = OrderedDict()
          GODict['BIOLOGICAL PROCESS'] = [element for element in BP]
          GODict['MOLECULAR FUNCTION'] = [element for element in MF]
          GODict['CELLULAR COMPONENT'] = [element for element in CC]
          GO_DF = pd.DataFrame(dict([ (k, pd.Series(v)) for k,v in GODict.items() ])) 
          table(ax, GO_DF)'''
          

## MesoconnectomePredictor class
### PreProcessing function
### author: Nestor Timonidis
### description:  
This function applies a set of pre-processing steps to the input gene expression and connectivity strength data, given a set of parameters. The steps are as follows:
*  1: Removal of structures dominated by NaN (not-a-number) values.
*  2: For gene expression data, imputation of the rest of NaN values by the median value of their corresponding column (feature) vector. For the connectivity data, sampling per column (feature) vector based on the frequency of zero elements and non-zero ones. A custom function called for the sampling is the BimodalImputation function that is described in the above cells.
*  3: Scaling both matrices by transforming them based on their cubic square root and then z-scoring them.
*  4: Removing outlier genes for the gene expression data based on their interquartile range. The function being called for that purpose is the RemoveOutliers function.
*  5: Binarizing the connectivity strength matrix based on the 66th percentile per column (source brain area).

* ### input: 
    * GeneExp = gene expression matrix.
    * ConStr = connectivity strength matrix.
    * params = information related to the pre-processing steps. Examples include removed structures (restCons, nanGenes) and gene acronyms. 
* ### output:
    * GeneExp_hat_ctr = Processed Gene Expression matrix, with a cubic square root transformation and z-scoring.
    * ConStr_sqrt_ctr = Processed Connectivity Strength matrix, with a cubic square root transformation and z-scoring.
    * y               = Binarized Conenctivity Strength matrix, based on the 66th percentile of the data per column.
    * sc_sqrt         = the scaler used to z-score the Connectivity Strength matrix.

In [19]:
class MesoconnectomePredictor(MesoconnectomePredictor):

    def PreProcessing(self,GeneExp, ConStr, params):
        
        # ************ Step 1: Cleaning the dataset from rows containing complete nans *******************#
        print 'Data cleaning phase ...\n'
        print 'Size before data cleaning ' + str(np.shape(ConStr))
        
        ConStr,nanCons = RemoveNanStructs(0,0.1).fit(ConStr)
        print 'Size after intermediate cleaning ' + str(np.shape(ConStr))
        params['restCons'] = [idx for idx in range(len(GeneExp)) if idx not in nanCons]
        GeneExp = np.delete(GeneExp,nanCons,0)
       
        GeneExp,nanGenes = RemoveNanStructs(0,0.03).fit(GeneExp)
        params['nanGenes'] = nanGenes; params['nanCons'] = nanCons
        params['str_cpy'] = np.delete(params['structure-abbrev'],params['nanCons'],0); params['str_cpy'] = np.delete(params['str_cpy'],params['nanGenes'],0)
        params['restGenes'] = [idx for idx in range(len(ConStr)) if idx not in nanGenes]
        ConStr = np.delete(ConStr,nanGenes,0)
        print 'Size after data cleaning ' + str(np.shape(ConStr))
        m = len(ConStr[0])
        tmp = [params['leaf_keys'][val] for val in params['restCons']]
        params['remaining_indices']   = [tmp[val] for val in params['restGenes']]
 
        # ************** Impute the value of the remaining nans with column averaging **************#
        print 'Data imputation phase ...\n'
        for task in range(len(ConStr[0])):
            nans = [val for val in ConStr[:,task] if mt.isnan(val) == True]
            if len(nans) > 0:
                ConStr[:,task] = BimodalImputation().fit(ConStr[:,task])

        GeneExp = Imputer(missing_values = 'NaN', strategy = 'median', axis = 0, verbose=0, copy=True).fit_transform(GeneExp)

        print 'Data normalization phase ....\n'
        
        GeneExp_hat = np.power(GeneExp, 1./3)
        GeneExp_hat_ctr = StandardScaler().fit_transform(GeneExp_hat)
        ConStr_sqrt = np.power(ConStr, 1./3)
        sc_sqrt = StandardScaler()
        sc_sqrt.fit(ConStr_sqrt)
        ConStr_sqrt_ctr = sc_sqrt.transform(ConStr_sqrt)

        # Remove outliers
        GeneExp_hat_ctr,rem_genes = RemoveOutliers().fit(GeneExp_hat_ctr)
        params['Gene Acronyms'] = np.delete(params['Gene Acronyms Original'],rem_genes,0);
        params['rem_genes'] = rem_genes
        print 'Removing outliers ... Reduced dim: '+ str(np.shape(GeneExp)) + ' ' + str(np.shape(ConStr))


        # Binarize the connectivity matrix for classification
        binThr  = 0.33333
        y       = np.zeros((np.shape(ConStr)))
        for j in range(ConStr.shape[1]):
            y[:,j] = BinarizeTheVector(ConStr[:,j],binThr)

        return GeneExp_hat_ctr, ConStr_sqrt_ctr, y, sc_sqrt

## MesoconnectomePredictor class
### LaminarRegistration function
### author: Nestor Timonidis
### description:  
Given a set of brain structure acronyms, this function scans the keywords for laminar-specific information and registers structures based on it.
* ### input: 
    * acronyms = brain structure acronyms.
* ### output:
    * laminar_profiles = layer specific profiles attributed to specific brain structures

In [20]:
def LaminarRegistration(acronyms):
 
    laminar_profiles = []
    for acro in acronyms:
        numbers = [idx for idx,val in enumerate(acro) if val.isdigit()]
        if numbers != []: 
            split_letter =  acro[numbers[0]-1]
            split_choices =  acro.split(split_letter)
            potential_choice = split_choices[len(split_choices)-1]
            blobs = [val for val in potential_choice if val.isdigit() == False]
            if len(blobs) < 2 and potential_choice != '' and 'r' not in potential_choice: 
                laminar_profiles.append('layer ' + potential_choice)
            elif '6a' in acro:
                laminar_profiles.append('layer 6a')
            else:
                laminar_profiles.append('layer inspecific')
        else:
            laminar_profiles.append('layer inspecific')
          
    return laminar_profiles   

## MesoconnectomePredictor class
### UnravelResults function
### author: Nestor Timonidis
### description:  
This function receives a ground truth vector and predictive results from a set of analyses 
and makes statistical evaluations by unraveling the results and comparing them with the ground truth.
* ### input:  
    * predictions = predictive results and ground truth from a set of analyses. 
    * params = information related to the predictive evaluation. Examples include the method category (classification or regression) and acronyms of genes used in the analysis.
    * scaler = model used to z-score the data before the predictive process. During the function restores the actual and predicted results into their original scale.
* ### output:
    * ClfResults = evaluations of the predictive results stored in a dictionary.

In [21]:
class MesoconnectomePredictor(MesoconnectomePredictor):
    
    def UnravelResults(self,predictions, params, scaler):
         
        num      = 1
        if len(predictions) == 1:
            predictions = predictions[0]
        if params['method'] == 'classification':
            y        = np.transpose([task[2] for task in predictions])
            y_preds  = np.transpose([task[1] for task in predictions])
            y_scores = np.asarray([task[0] for task in predictions])
            y_scores = y_scores.transpose(1,0,2)
        elif params['method'] == 'regression':
            if len(predictions) > 5:
                y        = np.transpose([task[2] for task in predictions])
                y_preds  = np.transpose([task[1] for task in predictions])
                y_scores = np.asarray([task[0] for task in predictions])
                if len(predictions[0]) > 4:
                    coeffs = [task[4] for task in predictions]
                mdls     = [task[3] for task in predictions]    
            else:
                y        = predictions[2]
                y_preds  = predictions[1]
                y_scores = predictions[0]
                mdls     = predictions[3]
                coeffs   = predictions[4]
                y_preds  = scaler.inverse_transform(y_preds)
                y        = scaler.inverse_transform(y)
                    
        if params['distance prior'] == True:
           y_preds  = np.asarray([MesoPred.DiscrimFunc(y_score, target, params) for target, y_score in enumerate(y_scores)]).transpose()

        ClfResults = {}
        ClfResults['Model Storage'] = mdls
        if params['method'] == 'classification':
            ClfResults['Precision'] = np.zeros((np.shape(y)[num],1))
            ClfResults['F-score']   = np.zeros((np.shape(y)[num],1))
            ClfResults['Recall']    = np.zeros((np.shape(y)[num],1))
            ClfResults['AURoc']     = np.zeros((np.shape(y)[num],1))
        elif params['method'] == 'regression':
            ClfResults['RMSE']      = np.zeros((np.shape(y)[num],1))
            ClfResults['r2']        = np.zeros((np.shape(y)[num],1))
      
        if len(predictions) > 4 and np.shape(coeffs)!= ():
        #if params['method'] == 'classification' and len(predictions[0][0]) > 4 and np.shape(coeffs[0])!= ():
            mean_coeffs = np.mean(coeffs,0)
            tmp_mean = np.argsort(mean_coeffs)[::-1]
            ClfResults['Mean Gene Scoring'] = [params['Gene Acronyms'][tmp_mean], mean_coeffs[tmp_mean], tmp_mean]
            ClfResults['Gene Scoring'] = []
            for coeff in coeffs:
                tmp = np.argsort(coeff)[::-1]
                ClfResults['Gene Scoring'].append([params['Gene Acronyms'][tmp],coeff[tmp], tmp])    
                
        '''elif params['method'] == 'regression' and len(predictions) > 4 and np.shape(coeffs)!= ():
            tmp = np.argsort(coeffs)[::-1]
            ClfResults['Gene Scoring'] = [params['Gene Acronyms'][tmp], coeffs[tmp], tmp]'''
       
        for num in range(len(y[0])):
            injection_task = 'Task ' + str(num)
            if params['method'] == 'classification':
                ClfResults['F-score'][num]   = metrics.f1_score(y[:,num], y_preds[:,num], \
                                                                average = 'binary')
                ClfResults['Precision'][num] = metrics.precision_score(y[:,num], \
                                                               y_preds[:,num],\
                                                               average = 'binary')
                ClfResults['Recall'][num]    = metrics.recall_score(y[:,num], y_preds[:,num], \
                                                             average = 'binary')
                ClfResults['AURoc'][num]     = metrics.roc_auc_score(y[:,num],y_scores[:,num][:,1],\
                                                             average = 'micro')
            elif params['method'] == 'regression':
                ClfResults['RMSE'][num]      = metrics.mean_squared_error(y[:,num], y_preds[:,num])
                ClfResults['r2'][num]        = metrics.r2_score(y[:,num], y_preds[:,num])

        return ClfResults

## Part I: Predictive Workflow

Initializing the parameters of the supervised learning models to be used.  
The models applied in this analysis are: 
*   classification = Random Forest classifier, Logistic Regression classifier
*   regression     = Random Forest regressor,  Ridge regressor

In [22]:
tol_num = 0.001

logreg_params = {'penalty': ['l2'], 'C' : np.logspace(-4, 1,base = 10), \
            'solver': ['newton-cg','lbfgs','sag','liblinear','saga'], \
            'class_weight' : ['balanced']}

rfc_params = {'max_depth': [10, 60, 100, None],
             'max_features': ['auto', 'sqrt'],
             'min_samples_leaf': [1, 2, 4],
             'min_samples_split': [2, 5, 10],
             'n_estimators': [25,50,100,200],\
             'criterion': ['gini'],
             'oob_score': [True], 'random_state' : [None],\
             'max_features' : ['auto'], 'class_weight' : ['balanced']}

ridge_params = {'alpha': [10e-4,10e-3,10e-2,0.5,1,2.5,5,7.5,10,100],
              'solver' : ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'saga'],
              'tol': [tol_num], 'max_iter' : [1000]}


sv_models = [GridSearchCV(LogisticRegression(), param_grid = logreg_params, scoring = 'f1', cv = 3, n_jobs = -1),
             GridSearchCV(RandomForestClassifier(), param_grid = rfc_params, scoring = 'f1', cv = 3, n_jobs = -1),
             DummyClassifier(),
             GridSearchCV(Ridge(), param_grid = ridge_params, 
                          scoring = 'neg_mean_squared_error', 
                          cv = 3, n_jobs = -1),
             RandomForestRegressor(n_estimators = 100),
             DummyRegressor()
            ]

Loading a series of data structures that will be used in the analysis.  
The data are being loaded for reasons of time efficiency.  
The Classes_and_Modules notebook provides comments and descriptions for the functions in which the data were created

In [23]:
leaves         = pk.load(open('leaf_nodes.pkl','rb'))
GeneExp        = h5py.File('G_Exp.hdf5', 'r')['dataset1']
GeneMeta       = pk.load(open('GeneMeta.pkl','r'))
GeneAcros      = np.asarray([val['genes'][0]['entrez_id'] for val in GeneMeta])
GeneNames      = np.asarray([val['genes'][0]['acronym'] for val in GeneMeta])
GeneMetaDict   = {val : GeneNames[idx] for idx,val in enumerate(GeneAcros)}
ConDict        = pk.load(open('CreLineDict.pkl','rb'))
folder_name    = 'Paper4/'
targetprofiles = OrderedDict()

Storage of information regarding the mouse brain structures.
The targetprofiles dictionary specializes in target areas 
with regards to connectivity patterns in the mouse mesoconnectome.
Its keys are: 
   *           str_acronym = acronyms of target areas/structures
   *           alt laminar profiles = laminar profiles of cortical structures
   *           father               = the brain regions in which each target belongs to      
   
   
The leaf_keys contain information about target areas that are present on the finest possible level of description.
For instance areas called VISp (primary visual area) and VISp l1 (primary visual area, layer 1) could both be found 
in the dataset, and therefore the leaf_keys are being used to filter out VISp as a non-leaf key in the structure hierarchy
since a finer description is present (VISp l1).

In [24]:
with open('acr2parent.json', 'r') as fp:
     acr2parent = json.load(fp)
with open('structures.csv','rb') as fp:
     structure = readtable(fp)
struct_idx_dict = {val:idx for idx,val in enumerate(structure['id'])}
targetprofiles['str_acronym'] = [val for val in structure['acronym']]
targetprofiles['alt laminar profiles'] = LaminarRegistration(targetprofiles['str_acronym'])
# storing layer 1 and layer 4 related areas for a latter analysis
l4_vals = [idx for idx,val in enumerate(targetprofiles['alt laminar profiles']) if val == 'layer 4']
l1_vals = [idx for idx,val in enumerate(targetprofiles['alt laminar profiles']) if val == 'layer 1']
lam_to_idx = {val:targetprofiles['alt laminar profiles'][idx] for idx,val in enumerate(structure['id'])}

# storage of leaf-level structures
leaf_keys = [int(val[0]) for val in leaves]  

for key in targetprofiles.keys():
    targetprofiles[key] = [val for idx,val in enumerate(targetprofiles[key]) if idx in leaf_keys]
targetprofiles['father'] = []
for strut in targetprofiles['str_acronym']:
    father = [val for key,val in acr2parent.items() if key == strut]
    if father == []:
       targetprofiles['father'].append(strut) 
    else:
       targetprofiles['father'].append(father[0])  

The params dictionary stores information which will be used throughout the analysis.
It will be passed as a parameter through the functions and updated at various steps. 

In [25]:
params = {'alter' : None, 'fetch' : 'all', 'distance prior': False,\
          'acronyms': targetprofiles['str_acronym'],\
          'method': 'classification', 'leaf': True, \
          'validation' : [StratifiedKFold(n_splits = 3, shuffle = True), KFold(n_splits = 3, shuffle = True)],\
          'prefix': 'Paper4/', 'primary models': [i for i in sv_models], \
          'Gene Acronyms Original': GeneAcros, 'Gene Metadata': GeneMetaDict,
          'leaf_keys' : leaf_keys}

Conversion of the Gene Expression dataset as a numpy array for analysis,  
initialization of various dictionaries to store preprocessed data
and results per tracer category (driver line),  
and initialization the the MesoconnectomePredictor for its
functions to be utilized

In [26]:
GeneExp = np.asarray(GeneExp)
rf_pred = {}; bl_pred = {}; ridge_pred = {}
ClfResults_ridge = {}; ClfResults_rf = {}; ClfResults_bl = {}
ConStr_proc = {}; GeneExp_proc = {}; y = {}

# matrix reduction to leaf-level structures
GeneExp = GeneExp[leaf_keys,:]
MesoPred = MesoconnectomePredictor(GeneExp, ConDict['wild_type']['ConMat'])

### Predictive Pipeline  
In our examples I utilize the unionized anatomical tract tracing data taken from the Allen Institute for Brain Science, as shown in the Class_and_Modules notebook. The data come in the form of a dictionary with nested matrices, where each dictionary corresponds to a tracer category like wild type or Syt6-Cre_KI148 cre-line experiments for instance.   
In order to demonstrate the MesoPred tool, a number of categories are selected as shown bellow. This analysis can take more than a day if run for all possible tracer categories. Therefore, for reasons of time efficiency one could skip this cell and move to the next one where the results can be loaded by stored pickle files.


In this pipeline a number of steps is being followed:  
0.  Selection of the tracer category (driver) of interest, reduction of tracing data to the leaf-level structures and storage of a number of parameters to be used in the analysis, such as the layer profiles and abbreviations of the target areas of the dataset.
1.  Preprocessing of the gene expression and tract tracing data. The function called in that step is PreProcessing
2.  Prediction with a logistic or linear ridge regression model using the cross-validation method for performance evaluation. The function called in that step is CustomCrossval.
3.  Prediction with a random forest classifier or regressor model using the cross-validation method for performance evaluation. The function called in that step is CustomCrossval.
4.  Prediction with a control (baseline) model using the cross-validation method for performance evaluation. The function called in that step is CustomCrossval.
5.  Storage of the predicted connectivity patterns, probability scores and gene importance (coefficient) scores per tracer in dictionaries stratified by the tested driver categories. The function called in that step is UnravelResults.
6.  Based on the stored connectivity patterns and the actual (ground truth) ones, evaluation of the predictive process with series of statistical measures. Examples include r squared and root mean squared error for regression, and f1-score and area under the roc curve for classification. The function called in that step is Evaluation.

In case that a set of results from a number of tracer categories have been stored,  
they can be loaded as shown bellow in order to save time.

In [27]:
ClfResults_rf = {}; ClfResults_bl = {}; rf_pred = {}; bl_pred = {}
for key in ConDict.keys():
    if os.path.isfile('./' + key +'_ridge_pred.pkl') == True and os.path.isfile('./' + key + '_all_results' + '_ridge.pkl') == True:
        tmp = pk.load(open('./' + key +'_ridge_pred.pkl','rb')) 
        ridge_pred[key] = [tmp[1],tmp[2]]
        ClfResults_ridge[key] = []#pk.load(open('./' + key + '_all_results' + '_ridge.pkl','rb')) 
    '''if os.path.isfile('./' + key +'_rf_pred.pkl') == True and os.path.isfile('./' + key + '_all_results' + '_rf.pkl') == True: 
        tmp = pk.load(open('./' + key +'_rf_pred.pkl','rb'))
        rf_pred[key] = [tmp[1],tmp[2]]
        ClfResults_rf[key] = pk.load(open('./' + key + '_all_results' + '_rf.pkl','rb'))      
    if os.path.isfile('./' + key +'_bl_pred.pkl') == True and os.path.isfile('./' + key + '_all_results' + '_bl.pkl') == True: 
        tmp = pk.load(open('./' + key +'_bl_pred.pkl','rb'))
        bl_pred[key] = [tmp[1],tmp[2]]
        ClfResults_bl[key] = pk.load(open('./' + key + '_all_results' + '_bl.pkl','rb'))  '''

### VISUALIZATION CHAPTER  
In this section, a number of steps are being followed   
for visualizing brain slices containing gene expression   
or connectivity information


Utilization of the MouseConnectivityCache (mcc) tool, provided by the Allen Institute for Brain Science Software Development Toolkit (SDK) at:  
https://alleninstitute.github.io/AllenSDK/_modules/allensdk/core/mouse_connectivity_cache.html  
With the mcc tool, the annotation volume of the mouse brain is being downloaded in both 25 and 10 micrometer resolution for visualization purposes.

In [ ]:
mcc_25 = MouseConnectivityCache(resolution = 25)
mcc_10 = MouseConnectivityCache(resolution = 10)
Annotation_25, annot_info = mcc_25.get_annotation_volume()
Annotation_10, annot_info = mcc_10.get_annotation_volume()

** Application of a set of experiment ids in the brain visualization tools (Brain Plotter) **

In [ ]:
# grab the StructureTree instance
structure_tree = mcc_10.get_structure_tree()
oapi           = OntologiesApi()
# Get isocortex summary structures
kid_to_father = {}
summary_structures = structure_tree.get_structures_by_set_id([688152357])
for val in summary_structures:
    kids = structure_tree.descendant_ids([val['id']])
    for kid in kids[0]:
        kid_to_father[kid] = val['id']

# For each distinct isocortical area, replace its children annotation with the area's id
father_acros_indices = {val['acronym']:val['id'] for val in summary_structures}

#store the original annotation shape because it will be flatten for computational simplicity when updating the parent nodes
orig_annot_shape = np.shape(Annotation_10)
flat_annot = np.ndarray.flatten(Annotation_10, order = 'C')

# Create a copy of the annotation structure that reduces areas to their parent nodes
Annotation_copy = np.zeros(np.shape(flat_annot))

for kid_id,parent_id in kid_to_father.items():
    tmp  = flat_annot == int(kid_id)
    nzero_ids = np.nonzero(tmp)
    Annotation_copy[nzero_ids] = parent_id 
     
Annotation_copy = np.ndarray.reshape(Annotation_copy, orig_annot_shape, order = 'C')    
    
pk.dump(Annotation_copy,open('Annotation_copy.pkl','wb'))

Selection of the 11th and 21st experiment for demonstration of the BrainPlotter tool

In [ ]:
tracer_for_selection = 'Ntsr1-Cre_GN220'
experiment_to_vis = [10,20]
UnionToVoxel_pred_25 = {}; UnionToVoxel_pred_10 = {}; UnionToVoxel_25 = {}; UnionToVoxel_10 = {}
struct_idx_dict_cp = struct_idx_dict.copy()
for key,value in struct_idx_dict.items():
    if value not in params['remaining_indices']:
       del struct_idx_dict_cp[key]
struct_idx_dict = struct_idx_dict_cp    
Annotation_copy = pk.load(open('Annotation_copy.pkl','rb'))

In [ ]:
for j in experiment_to_vis:
    
    key_to_save = tracer_for_selection + '_' + str(j)
    exp_id = ConDict[tracer_for_selection]['id'][j]
    ConStr = ConDict[tracer_for_selection]['ConMat']
    l4_vol  = np.nanmean(ConStr[l4_vals,j]); l1_vol  = np.nanmean(ConStr[l1_vals,j])
    Pred_ConStr = ridge_pred[tracer_for_selection]
    
    #UnionToVoxel_25[key_to_save], UnionToVoxel_pred_25[key_to_save] = \
    #                   FromUnion2Voxel(Annotation_25, struct_idx_dict, Pred_ConStr[2][:,j], Pred_ConStr[1][:,j])
    # pk.dump(UnionToVoxel_25[key_to_save], open('UnionToVoxel_25_for_' + str(key_to_save),'wb'))
    #UnionToVoxel_10[key_to_save], UnionToVoxel_pred_10[key_to_save] = \
    #                   FromUnion2Voxel(Annotation_10, struct_idx_dict, Pred_ConStr[2][:,j], Pred_ConStr[1][:,j])
    # pk.dump(UnionToVoxel_10[key_to_save], open('UnionToVoxel_10_for_' + str(key_to_save),'wb'))        
    
    #BP_10 = BrainPlotter(mcc_10, experiment_id = exp_id, res = 10, driver = tracer_for_selection + str(exp_id))
    '''BP_25 = BrainPlotter(mcc_25, experiment_id = exp_id, res = 25, driver = tracer_for_selection + str(exp_id))
    
    BP_25.plot_slice(struct_idx_dict, 'None', Annotation_25, matrix = UnionToVoxel_25[key_to_save],\
                     savefile = str(tracer_for_selection) + '_union_' + str(j) + '.png')
    BP_25.plot_slice(struct_idx_dict, 'None', Annotation_25, matrix = UnionToVoxel_pred_25[key_to_save],\
                     savefile = str(tracer_for_selection) + '_union_pred_' + str(j) + '.png')
    '''
    BP_10.plot_flatmap('None', Annotation_copy, matrix = UnionToVoxel_10[key_to_save],\
                       savefile = str(tracer_for_selection) + '_union_' + str(j) + '_flatmap')
    BP_10.plot_flatmap('None', Annotation_copy, matrix = UnionToVoxel_pred_10[key_to_save],\
                       savefile = str(tracer_for_selection) + '_union_pred_' + str(j) + '_flatmap')
    
    print 'layer 4 volume ' + str(l4_vol) + '\nlayer 1 volume ' + str(l1_vol)
   

### Part IV: Link to Scalable Brain Atlas Composer Section

Parsing results to json format and Calling the SBA API service to visually inspect them

In [ ]:
JSonDict = {'bas': {'provider': "sba",
             'atlas': "ABA_v3",
             'orientation': "RAS",
             'unit': "mm"},
             'style': {}, 'markers':[]} 


In [ ]:
for tracer_name in ridge_pred.keys():
    if tracer_name == 'wild_type':
       tracer_layer = ''
       tracer_cell  = ''
    else:
       tracer_layer = ConDict[tracer_name]['layer'][0]
       tracer_cell =  ConDict[tracer_name]['cell-type'][0]
    for idx, measure in enumerate(ClfResults_ridge[tracer_name]['r2']):
        JSonDict['markers'].append({})
        JSonDict['markers'][len(JSonDict['markers'])-1]['value']    = str(measure[0])
        JSonDict['markers'][len(JSonDict['markers'])-1]['position'] = [str(coo) for coo in ConDict[tracer_name]['Coordinates'][idx]]
        JSonDict['markers'][len(JSonDict['markers'])-1]['region']   = ConDict[tracer_name]['structure-abbrev'][idx]
        JSonDict['markers'][len(JSonDict['markers'])-1]['relsize']  = "0.5"
        JSonDict['markers'][len(JSonDict['markers'])-1]['cre-line'] = tracer_name
        if 'layer' in ConDict[tracer_name]:
            JSonDict['markers'][len(JSonDict['markers'])-1]['layer']  = ConDict[tracer_name]['layer'][idx] 
 

In [ ]:
from IPython.display import display, Javascript, clear_output

# SBA Composer page to load: here it is the -dev version and atlas ABA_v3
sbaHost = 'https://scalablebrainatlas.incf.org/composer-dev';
sbaUrl = sbaHost+'/?template=ABA_v3';

class sbaInterface_class:
    def __init__(this,sbaUrl):
        # In Javascript, load sbaInterface.js and create a new sbaInterface object
        display(Javascript("""
            var script = document.createElement('script');
            script.src = '{}/../js/sba-interface.js';
            script.onload = function() {{
              window.global_sbaInterface = new sbaInterface_class('{}');
            }}
            document.head.appendChild(script)
        """.format(sbaHost,sbaUrl)))
        clear_output()
        
    def send(this,sbaCommand):
        display(Javascript("""window.global_sbaInterface.send({})""".format(sbaCommand)))
        clear_output() # prevents clogging the Jupyter notebook

sbaInterface = sbaInterface_class(sbaUrl)

Utilization of the sbaInterface_class object to <br>
visually inspect the MesoPred results with the <br>
use of the 3D brain visualization tool <br>
'Scalable Brain Atlas Composer' <br>

In [ ]:
#sbaInterface.send(json.dumps({'method':'Composer.scatter3d' ,'params' : JSonDict}))
sbaCommand = {
    'method':'Composer.scatter3d' ,
    'params' : JSonDict
}
sbaCommand = json.dumps(sbaCommand)
sbaInterface.send(sbaCommand)

After the prediction of connectivity patterns with the training of unionized data has been evaluated at the plotting phase,
the next and final step is the visual inspection of the connectivity prediction for the voxelized gridded data for which
there is no ground truth information. The predicted results for the models constructed with the Logistic Regression classification
algorithms have been selected as an example, and are being given to the Save2Nifti function. In this function, 
nifti volume files are being produced with the connectivity patterns for various target brain areas. The nifti volumes  converted as urls can be given to the SBA composer as input for visual inspection, after the user has clicking on the provided link/s.

In [ ]:
nifti_urls = Save2Nifti(UnionToVoxel_10)

Now we can select indicative target areas and visually inspect the predicted connectivity patterns between the area and all voxels in the 3D volume space.
6 indicative examples are selected ('VISpl', 'OT', 'MPO', 'IPN', 'CEA', 'BST'):


In [ ]:
for key in nifti_urls.keys():
    display(HTML('<form target="SBA COMPOSER" action="https://scalablebrainatlas.incf.org/composer-dev/index.php?template=ABA_v3" method="post"><input type="hidden" name="scene" value="{}">Predicted connectivity patterns between 3D brain volume and area {} <input type="submit" value="Show connections to {} in 3D"></form>'.format(nifti_urls[key].replace('"','&quot;'),key,key)))

**As a side note**: The plane corresponding to the brain slice at the SBA composer is not fixed but can be rotated with the use of the left click or zoomed with the right click. Moreover, the toolbar at the left of the screen provides to the user a set of visualization configurations.
Furthermore in terms of predictive results, the black color at the SBA composer represents a lack of connection between the voxels and the corresponding target area, while the white color represents connection between them.

In [ ]:
Clearing storage space from the nifti volumes

In [ ]:
call(['rm','*.nii.gz'])